In [ ]:
import heapq as hq

class Passenger:
    def __init__(self, BM, exit_time):
        self.bm = BM
        self.exit = exit_time
    def __lt__(self, other):
        return (self.exit, self.bm) < \
               (other.exit, other.bm)

class BM:
    def __init__(self, passenger, capacity, stop, itinerary):
        self.passenger = passenger
        self.capacity = capacity
        self.stop = stop
        self.itinerary = itinerary
    def __le__(self, other):
        pass
    
class Event:
    def __init__(self, time, action, bm):
        self.time = time
        self.action = action
        self.bm = bm
    def __lt__(self, other):
        return (self.time) < (other.time)
    def __le__(self, other):
        return (self.time) <= (other.time)

In [ ]:
passengers = [[Passenger(0,120)]*5, [Passenger(0,120)]*2]
bms = [BM(0,3,0,[0,1])]
event0 = Event(0,'load',0) 
eventq = []
hq.heappush(eventq, event0)


while eventq:
    event = hq.heappop(eventq)
    print(event)
    bahn = bms[event.bm]
    curent_node = bahn.itinerary [bahn.stop % len(bahn.itinerary)]
    next_node =  bahn.itinerary [(bahn.stop +1)  % len(bahn.itinerary)]
    print(curent_node)
    print(next_node)
    if event.action == 'load':
        #free = bahn.capacity - bahn.passenger
        while bahn.passenger < bahn.capacity:
            waiting = [idx for idx, passenger in enumerate(passengers[curent_node]) if passenger.bm == event.bm]
            print(waiting)
            if not waiting:
                break
            passenger = passengers[curent_node].pop(waiting[0])
            print(passenger)
            bahn.passenger += 1
            hq.heappush(eventq, Event(event.time + passenger.exit,'exit', event.bm))
            print(eventq)
        hq.heappush(eventq, Event(event.time+150, 'load', event.bm))
        print(eventq)
        bahn.stop = next_node
    elif event.action == 'exit':
        bm.passenger -= 1
    else:
        raise Exception('Unknown event action')

In [ ]:
passengers = [[{'bm': 0, 'exit_time': 120}]*10, [{'bm': 0, 'exit_time': 120}]*8]
bms = [{'n_passengers': 0, 'capacity': 3, 'stop': 0, 'itinerary': [0, 1]}]

event0 = Event(0,'load',0) 
eventq = []
hq.heappush(eventq, event0)

while eventq:
    event = hq.heappop(eventq)
    bm = bms[event.bm]
    current_node = bm['itinerary'][bm['stop'] % len(bm['itinerary'])]
    next_node = bm['itinerary'][(bm['stop']+1) % len(bm['itinerary'])]
    if event.action == 'load':
        free = bm['capacity'] - bm['n_passengers']
        while bm['n_passengers'] < bm['capacity']:
            waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger['bm'] == event.bm]
            print(waiting)
            if not waiting:
                break
            passenger = passengers[current_node].pop(waiting[0])
            print(passenger)
            bm['n_passengers'] += 1
            hq.heappush(eventq, Event(event.time + passenger['exit_time'],'exit', event.bm))
            #hq.heappush(eventq, {'time': event['time'] + passenger['exit_time'], 'action': 'exit', 'bm': event['bm']})
            print(eventq)
        hq.heappush(eventq, Event(event.time+150, 'load', event.bm))
        #hq.heappush(eventq, {'time': event['time']+150, 'action': 'load', 'bm': event['bm']})
        bm['stop'] += 1
        # Generate events for exiting passengers
    elif event.action == 'exit':
        bm['n_passengers'] -= 1
    else:
        raise Exception('Unknown event action')